In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# make deterministic
from mingpt.utils import set_seed
set_seed(44)

In [3]:
import os
import math
import time
from tqdm import tqdm
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from data.othello import Othello, OthelloBoardState, permit_reverse
from mingpt.dataset import CharDataset
from mingpt.utils import sample
from mingpt.model import GPT, GPTConfig, GPTforProbing
from mingpt.trainer import Trainer, TrainerConfig
from mingpt.probe_model import BatteryProbeClassification, BatteryProbeClassificationTwoLayer

In [8]:
# loading probes
exp = "state_tl256_e40"
probes = {}
for layer in range(1, 9):
    p = BatteryProbeClassificationTwoLayer(torch.cuda.current_device(), probe_class=3, num_task=64, mid_dim=256)
    load_res = p.load_state_dict(torch.load(f"./ckpts/grok/probes/{exp}/layer{layer}/checkpoint.ckpt"))
    p.eval()
    probes[layer] = p

In [ ]:
othello = Othello(n_games=100, deduplicate=False)
train_dataset = CharDataset(othello)

mconf = GPTConfig(61, 59, n_layer=8, n_head=8, n_embd=512)
model = GPTforProbing(mconf)

In [7]:
# summarizing probe loss

import json

root = f"ckpts/bias/probes/state_tl256_b95"
errs = []
for i in range(8):
    name = f"layer{i+1}/tensorboard.txt"
    with open(f"{root}/{name}", "r") as file:
        j = json.load(file)
        test_acc = j['test_acc_cont']
        err = 100 * (1 - test_acc[-1])
        errs.append(err)
        # print(f"layer {i+1} error rate: {err:.5f}")

print(", ".join([str(e) for e in errs]))

20.731401647594826, 17.801225762873486, 16.90723339160839, 17.143346710108077, 17.491358073744433, 18.251698585505405, 19.345117477219752, 21.353305785123965
